In [1]:
import numpy as np
from skfem import (
    MeshTri, Basis, FacetBasis, ElementTriP2, ElementTriP1, ElementVector,
    asm, bmat, condense, solve, BilinearForm, LinearForm, Mesh, ElementTetP2, ElementTetP1
)
from skfem.utils import  solver_iter_krylov
from skfem.models.general import divergence
from skfem.models.poisson import vector_laplace
from skfem.helpers import grad, dot, laplacian
from skfem.helpers import laplacian, precompute_operators
import matplotlib.pyplot as plt

import scipy.sparse as sp
from scipy.sparse.linalg import eigsh, eigs, splu, LinearOperator

import scipy.linalg
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

import plotly.graph_objects as go
from skfem.mesh import MeshTet # MeshTet para mallas tetraédricas (3D)

## Importar malla y etiquetar boundaries

In [2]:
mesh = Mesh.load("tubo.msh");

In [3]:
def get_boundary_facets_indices(mesh): 
    boundary_facets_idx = mesh.boundary_facets()
    facets  = mesh.facets
    nodes   = mesh.p
    
    pared_idx   = []
    salida_idx  = []
    entrada_idx = []

    for e in boundary_facets_idx:
        coords_facets = nodes[:, facets[:, e]]
        m = np.mean(coords_facets, axis=1) # m = [mean(x), mean(y), mean(z)]

        # Lógica de Etiquetado (usando Y = m[1] como eje vertical, cerca de -2)
        
        # Pared: Partes que no son la entrada/salida (e.g., Y > -2.0)
        if m[1] > -2.0:
            pared_idx.append(e)
            
        # Salida: Extremo derecho (Y en -2.0 y X > 0)
        elif m[1]==-2.0 and m[0] > 0.0:
            salida_idx.append(e)
            
        # Entrada: Extremo izquierdo (Y en -2.0 y X < 0)
        elif m[1]==-2.0 and m[0] < 0.0:
            entrada_idx.append(e)
            
    return np.array(pared_idx), np.array(salida_idx), np.array(entrada_idx)

wall, outflow, inflow = get_boundary_facets_indices(mesh)

In [4]:
# Graficar malla y boundary facets
facets = mesh.facets
nodes  = mesh.p


def get_mesh_data(indices_facets):  
    if indices_facets.size == 0:
        return None
        
    # Extraer las facetas que corresponden a los índices calculados
    facet_indices = facets[:, indices_facets]
    
    x = nodes[0, :]
    y = nodes[1, :]
    z = nodes[2, :]
    
    i = facet_indices[0, :]
    j = facet_indices[1, :]
    k = facet_indices[2, :]
    
    return x, y, z, i, j, k


traces = []

# Wall
wall_data = get_mesh_data(wall)
if wall_data:
    x, y, z, i, j, k = wall_data
    traces.append(
        go.Mesh3d(
            x=x, y=y, z=z, i=i, j=j, k=k,
            color='rgba(0, 200, 0, 0.5)',
            opacity=0.5,
            name='Wall',
            showlegend=True,
            lighting=dict(ambient=0.5, diffuse=0.5, specular=0.8)
        )
    )

# Inflow
inflow_data = get_mesh_data(inflow)
if inflow_data:
    x, y, z, i, j, k = inflow_data
    traces.append(
        go.Mesh3d(
            x=x, y=y, z=z, i=i, j=j, k=k,
            color='rgba(200, 0, 0, 0.8)',
            opacity=0.8,
            name='Inflow',
            showlegend=True,
            lighting=dict(ambient=0.5, diffuse=0.5, specular=0.8)
        )
    )

# Outflow
outflow_data = get_mesh_data(outflow)
if outflow_data:
    x, y, z, i, j, k = outflow_data
    traces.append(
        go.Mesh3d(
            x=x, y=y, z=z, i=i, j=j, k=k,
            color='rgba(0, 0, 200, 0.8)',
            opacity=0.8,
            name='Outflow',
            showlegend=True,
            lighting=dict(ambient=0.5, diffuse=0.5, specular=0.8)
        )
    )

# Nodos de la malla
traces.append(
    go.Scatter3d(
        x=nodes[0, :],
        y=nodes[1, :],
        z=nodes[2, :],
        mode='markers',
        marker=dict(
            size=1,
            color=nodes[2, :],
            opacity=0.8
        ),
        name='Nodos Malla',
        showlegend=True
    )
)

fig = go.Figure(data=traces)

fig.update_layout(
    title='Malla 3D Interactiva de Fronteras de Tubería en U',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()


## Navier stokes 3D

In [5]:
# Definir elementos y bases (P2 para velocidad, P1 para presión) en 3D
element = {
    'u': ElementVector(ElementTetP2(), dim=3),
    'p': ElementTetP1(),
}
basis = {
    'u': Basis(mesh, element['u'], intorder=4),
    'p': Basis(mesh, element['p'], intorder=4),
}
basis_u, basis_p = basis['u'], basis['p']
Nu, Np = basis_u.N, basis_p.N
N      = Nu + Np

In [6]:
mu    = 0.035 # viscosidad dinámica
rho   = 1.0   # densidad
nu    = mu / rho # Viscosidad cinemática
V_car = 1.0    # Velocidad característica
L_car = 1.0    # Longitud característica
Re    = V_car * L_car / nu

In [7]:
@BilinearForm
def mass_matrix(u, v, w):
    return dot(u, v)

# Término convectivo linealizado
@BilinearForm
def convection(u, v, w):
    advection_field = w['w']
    grad_u = grad(u)
    return np.einsum('j...,ij...,i...->...', advection_field, grad_u, v)


# Ensamblaje de matrices
A =  asm(vector_laplace, basis_u)               
B = -asm(divergence, basis_u, basis_p)   

F_u = basis['u'].zeros()
F_p = basis['p'].zeros()
F = np.hstack([F_u, F_p])

In [8]:
# DOFs de las fronteras
dofs_wall   = basis_u.get_dofs(wall).all()
dofs_outflow  = basis_u.get_dofs(outflow).all()
dofs_inflow = basis_u.get_dofs(inflow).all()

dofs_inflow_x = dofs_inflow[dofs_inflow % 3 == 0]
dofs_inflow_y = dofs_inflow[dofs_inflow % 3 == 1]
dofs_inflow_z = dofs_inflow[dofs_inflow % 3 == 2]
dofs_p_salida = basis_p.get_dofs(outflow).all()

xin = basis_u.doflocs[0, dofs_inflow_x]
yin = basis_u.doflocs[1, dofs_inflow_y]
zin = basis_u.doflocs[2, dofs_inflow_z]

In [9]:
# Inflow function
def inflow_velocity(x, y, z):
    x_min = np.min(xin)
    x_max = np.max(xin)
    z_min = np.min(zin)
    z_max = np.max(zin)
    # Velocity components: flow in y-direction
    return (x - x_min) * (x_max - x) * (z - z_min) * (z_max - z)

v_inflow = inflow_velocity(xin, yin, zin)

In [10]:
x_boundaries = np.zeros(Nu+Np)

# Inflow
x_boundaries[dofs_inflow_x] = 0.0
x_boundaries[dofs_inflow_y] = v_inflow/np.max(v_inflow)
x_boundaries[dofs_inflow_z] = 0.0

# Presión a la salida
x_boundaries[Nu + dofs_p_salida[0]] = 0.0

D_all = np.concatenate([
    basis_u.get_dofs(wall).all(),
    basis_u.get_dofs(inflow).all(),
    Nu+np.array([dofs_p_salida[0]])
]) 

In [11]:
# Visualizar la condición de inflow en 3D
fig_inflow = go.Figure()

# Crear scatter plot de los nodos de inflow coloreados por la velocidad
fig_inflow.add_trace(
    go.Scatter3d(
        x=xin,
        y=yin,
        z=zin,
        mode='markers',
        marker=dict(
            size=5,
            color=v_inflow/np.max(v_inflow),
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Velocidad"),
            cmin=0,
            cmax=1.0
        ),
        name='Velocidad Inflow',
        text=[f'v={v:.4f}' for v in v_inflow],
        hovertemplate='x: %{x:.3f}<br>y: %{y:.3f}<br>z: %{z:.3f}<br>%{text}<extra></extra>'
    )
)

# Agregar también la malla de inflow para contexto
if inflow_data:
    x_mesh, y_mesh, z_mesh, i_mesh, j_mesh, k_mesh = inflow_data
    fig_inflow.add_trace(
        go.Mesh3d(
            x=x_mesh, y=y_mesh, z=z_mesh,
            i=i_mesh, j=j_mesh, k=k_mesh,
            color='rgba(200, 0, 0, 0.3)',
            opacity=0.3,
            name='Superficie Inflow',
            showlegend=True
        )
    )

fig_inflow.update_layout(
    title='Condición de Velocidad en Inflow (Perfil Parabólico 3D)',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    showlegend=True
)

fig_inflow.show()


### Stokes

In [37]:
# Calcular solución inicial de stokes
K_stokes = bmat([[mu * A,     B.T],  
                 [B,         None]], format='csr')

sol0 = solve(*condense(K_stokes, F, D=D_all, x=x_boundaries))
u_ref = sol0[:Nu].copy()
p_ref = sol0[Nu:Nu+Np].copy()
ux = u_ref[0::3]
uy = u_ref[1::3]
uz = u_ref[2::3]

In [13]:
P1_basis = Basis(mesh, ElementTetP1())

# Mapeo de Presión (Escalar)
pressure_scalar = p_ref 

# Mapeo de Velocidad (Vectorial 3D)
velocity_vector = np.zeros((mesh.nvertices, 3))

for idx in range(mesh.nvertices):
    velocity_vector[idx, 0] = u_ref[basis_u.nodal_dofs[0, idx]]  # componente X
    velocity_vector[idx, 1] = u_ref[basis_u.nodal_dofs[1, idx]]  # componente Y
    velocity_vector[idx, 2] = u_ref[basis_u.nodal_dofs[2, idx]]  # componente Z

point_data_to_export = {
    "velocity": velocity_vector,
    "pressure": pressure_scalar,
}

mesh.save(
        "stokes3d_mapped.vtu", 
        point_data=point_data_to_export
)

In [43]:
help(mesh.save)

Help on method save in module skfem.mesh.mesh:

save(
    filename: Union[str, os.PathLike],
    point_data: Optional[Dict[str, numpy.ndarray]] = None,
    cell_data: Optional[Dict[str, numpy.ndarray]] = None,
    **kwargs
) -> None method of skfem.mesh.mesh_tet_1.MeshTet1 instance
    Export the mesh and fields using meshio.

    Parameters
    ----------
    filename
        The output filename, with suffix determining format;
        e.g. .msh, .vtk, .xdmf
    point_data
        Data related to the vertices of the mesh.
    cell_data
        Data related to the elements of the mesh.



#### strong error

In [38]:
### Error forma fuerte
basis_x = basis_u.split_bases()[0]
basis_y = basis_u.split_bases()[1]
basis_z = basis_u.split_bases()[2]
u_x = u_ref[0::3]
u_y = u_ref[1::3]
u_z = u_ref[2::3] 
p_sol = p_ref 

edofs_x, phix, grad_phix, laplacian_phix = precompute_operators(basis_x)
edofs_y, phiy, grad_phiy, laplacian_phiy = precompute_operators(basis_y)
edofs_z, phiz, grad_phiz, laplacian_phiz = precompute_operators(basis_z)
edofs_p, phip, grad_phip, laplacian_phip = precompute_operators(basis_p)

In [41]:
# Velocidad y presión en puntos de cuadratura
u_val = np.einsum('dq, de -> qe', phix, u_x[edofs_x]) 
v_val = np.einsum('dq, de -> qe', phiy, u_y[edofs_y])
w_val = np.einsum('dq, de -> qe', phiz, u_z[edofs_z])
p_val = np.einsum('dq, de -> qe', phip, p_sol[edofs_p])

# Calcular operadores en coordenadas físicas
grad_u_fisico_x = np.einsum('de, diqe -> iqe', u_x[edofs_x]  , grad_phix)
grad_u_fisico_y = np.einsum('de, diqe -> iqe', u_y[edofs_y]  , grad_phiy)
grad_u_fisico_z = np.einsum('de, diqe -> iqe', u_z[edofs_z]  , grad_phiz)
grad_p_fisico   = np.einsum('de, diqe -> iqe', p_sol[edofs_p], grad_phip)
laplacian_u_x   = np.einsum('de, de   -> e'  , u_x[edofs_x]  , laplacian_phix)
laplacian_u_y   = np.einsum('de, de   -> e'  , u_y[edofs_y]  , laplacian_phiy)
laplacian_u_z   = np.einsum('de, de   -> e'  , u_z[edofs_z]  , laplacian_phiz)

In [ ]:
# Residuo momentum en X:
res_momentum_x = np.abs(grad_p_fisico[0] - (laplacian_u_x)*mu)

# Residuo momentum en Y:
res_momentum_y = np.abs(grad_p_fisico[1] - (laplacian_u_y)*mu)

# Residuo momentum en Z: 
res_momentum_z = np.abs(grad_p_fisico[2] - (laplacian_u_z)*mu)

# Residuo Continuidad:
res_continuity = np.abs(grad_u_fisico_x[0] + grad_u_fisico_y[1] + grad_u_fisico_z[2])

print(f"Residuo momentum en X - min: {np.min(res_momentum_x):.4e}, max: {np.max(res_momentum_x):.4e}")
print(f"Residuo momentum en Y - min: {np.min(res_momentum_y):.4e}, max: {np.max(res_momentum_y):.4e}")
print(f"Residuo momentum en Z - min: {np.min(res_momentum_z):.4e}, max: {np.max(res_momentum_z):.4e}")
print(f"Residuo continuidad - min: {np.min(res_continuity):.4e}, max: {np.max(res_continuity):.4e}")


Residuo momentum en X - min: 3.2783e-06, max: 1.5782e+01
Residuo momentum en Y - min: 8.6663e-08, max: 2.0197e+01
Residuo momentum en Z - min: 8.1038e-08, max: 1.5371e+01
Residuo continuidad - min: 3.8204e-07, max: 1.0100e+01


### Navier-Stokes

In [12]:
def solve_ns_picard(u_init, p_init, Re, max_iter, tol):
    u = u_init
    p = p_init
    for it in range(max_iter):
        # Campo de advección congelado w := u^(it) en puntos de cuadratura
        W = basis_u.interpolate(u)   

        # Ensambla bloque convectivo C(w)
        C = asm(convection, basis_u, w=W)

        # Matriz bloque del paso linealizado
        K = bmat([[(1/Re) * A + C, B.T ],
                  [B,              None]], format='csr')

        # Resolver
        sol = solve(*condense(K, F, D=D_all, x=x_boundaries))
        u_new = sol[:Nu]
        p_new = sol[Nu:Nu+Np]

        # Criterio de convergencia
        du = u_new - u
        rel_u = np.linalg.norm(du) / (np.linalg.norm(u_new) + 1e-16)

        dp = p_new - p
        rel_p = np.linalg.norm(dp) / (np.linalg.norm(p_new) + 1e-16)

        # # Sub-relajación si se desea
        u = u_new
        p = p_new

        if rel_u < tol and rel_p < tol:
            print(f"Convergió en {it+1} iteraciones, residuo {max(rel_u, rel_p):.4e}")
            return u, p, True
    print("No convergió en el número máximo de iteraciones")
    return u, p, False

In [49]:
# Calcular solución inicial de stokes
K_stokes = bmat([[mu * A,     B.T],  
                 [B,         None]], format='csr')
A_stokes, F_stokes, xI, I = condense(K_stokes, F, D=D_all, x=x_boundaries)
sol0 = solve(*condense(K_stokes, F, D=D_all, x=x_boundaries))
u_ref = sol0[:Nu]
p_ref = sol0[Nu:Nu+Np]
ux = u_ref[0::3]
uy = u_ref[1::3]
uz = u_ref[2::3]

In [52]:
from skfem.utils import solver_iter_krylov
from scipy.sparse.linalg import LinearOperator, minres


In [ ]:
# solve(A_stokes, F_stokes,
#             solver=solver_iter_krylov(minres),
#             M=build_pc_diag(asm(mass, basis['p'])))

In [48]:
# Resolver incrementanto Re
Re = 1
Re_linspace = np.linspace(1, Re, 1)

for R in Re_linspace:
    print(f"Resolviendo para Re = {R:.2f}")
    u_ref, p_ref, flag = solve_ns_picard(u_ref, p_ref, R, max_iter=500, tol=1e-12)

    if not flag:
        print(f"No se pudo converger para este Re = {R:.2f}.")
        break

u_sol = u_ref
p_sol = p_ref


Resolviendo para Re = 1.00
Convergió en 7 iteraciones, residuo 1.4050e-13


In [18]:
P1_basis = Basis(mesh, ElementTetP1())

# Mapeo de Presión (Escalar)
pressure_scalar = p_sol 

# Mapeo de Velocidad (Vectorial 3D)
velocity_vector = np.zeros((mesh.nvertices, 3))

for idx in range(mesh.nvertices):
    velocity_vector[idx, 0] = u_sol[basis_u.nodal_dofs[0, idx]]  # componente X
    velocity_vector[idx, 1] = u_sol[basis_u.nodal_dofs[1, idx]]  # componente Y
    velocity_vector[idx, 2] = u_sol[basis_u.nodal_dofs[2, idx]]  # componente Z

point_data_to_export = {
    "velocity": velocity_vector,
    "pressure": pressure_scalar,
}

mesh.save(
        "navierstokes3d_mapped.vtu", 
        point_data=point_data_to_export
)

### strong error

In [17]:
### Error forma fuerte
basis_x = basis_u.split_bases()[0]
basis_y = basis_u.split_bases()[1]
basis_z = basis_u.split_bases()[2]
u_x = u_ref[0::3]
u_y = u_ref[1::3]
u_z = u_ref[2::3] 
p_sol = p_ref 

edofs_x, phix, grad_phix, laplacian_phix = precompute_operators(basis_x)
edofs_y, phiy, grad_phiy, laplacian_phiy = precompute_operators(basis_y)
edofs_z, phiz, grad_phiz, laplacian_phiz = precompute_operators(basis_z)
edofs_p, phip, grad_phip, laplacian_phip = precompute_operators(basis_p)

In [ ]:
# Velocidad y presión en puntos de cuadratura
u_val = np.einsum('dq, de -> qe', phix, u_x[edofs_x]) 
v_val = np.einsum('dq, de -> qe', phiy, u_y[edofs_y])
w_val = np.einsum('dq, de -> qe', phiz, u_z[edofs_z])
p_val = np.einsum('dq, de -> qe', phip, p_sol[edofs_p])

# Calcular operadores en coordenadas físicas
grad_u_fisico_x = np.einsum('de, diqe -> iqe', u_x[edofs_x]  , grad_phix)
grad_u_fisico_y = np.einsum('de, diqe -> iqe', u_y[edofs_y]  , grad_phiy)
grad_u_fisico_z = np.einsum('de, diqe -> iqe', u_z[edofs_z]  , grad_phiz)
grad_p_fisico   = np.einsum('de, diqe -> iqe', p_sol[edofs_p], grad_phip)
laplacian_u_x   = np.einsum('de, de|  -> e'  , u_x[edofs_x]  , laplacian_phix)
laplacian_u_y   = np.einsum('de, de   -> e'  , u_y[edofs_y]  , laplacian_phiy)
laplacian_u_z   = np.einsum('de, de   -> e'  , u_z[edofs_z]  , laplacian_phiz)

In [23]:
# Residuo momentum en X:

res_momentum_x = np.abs(grad_p_fisico[0] - (laplacian_u_x)/Re + (u_val * grad_u_fisico_x[0] + v_val * grad_u_fisico_x[1] + w_val * grad_u_fisico_x[2]) )

# Residuo momentum en Y:
res_momentum_y = np.abs(grad_p_fisico[1] - (laplacian_u_y)/Re + (u_val * grad_u_fisico_y[0] + v_val * grad_u_fisico_y[1] + w_val * grad_u_fisico_y[2]) )

# Residuo momentum en Z: 
res_momentum_z = np.abs(grad_p_fisico[2] - (laplacian_u_z)/Re + (u_val * grad_u_fisico_z[0] + v_val * grad_u_fisico_z[1] + w_val * grad_u_fisico_z[2]) )

# Residuo Continuidad:
res_continuity = np.abs(grad_u_fisico_x[0] + grad_u_fisico_y[1] + grad_u_fisico_z[2])

print(f"Residuo momentum en X - min: {np.min(res_momentum_x):.4e}, max: {np.max(res_momentum_x):.4e}")
print(f"Residuo momentum en Y - min: {np.min(res_momentum_y):.4e}, max: {np.max(res_momentum_y):.4e}")
print(f"Residuo momentum en Z - min: {np.min(res_momentum_z):.4e}, max: {np.max(res_momentum_z):.4e}")
print(f"Residuo continuidad - min: {np.min(res_continuity):.4e}, max: {np.max(res_continuity):.4e}")


Residuo momentum en X - min: 6.8784e-06, max: 4.5227e+02
Residuo momentum en Y - min: 1.0013e-07, max: 5.7956e+02
Residuo momentum en Z - min: 1.1304e-06, max: 4.4053e+02
Residuo continuidad - min: 1.9370e-07, max: 1.0095e+01


In [20]:
# Calcular el máximo error por elemento para cada tipo de residuo
max_res_momentum_x_per_elem = np.max(res_momentum_x, axis=0)
max_res_momentum_y_per_elem = np.max(res_momentum_y, axis=0)
max_res_momentum_z_per_elem = np.max(res_momentum_z, axis=0)
max_res_continuity_per_elem = np.max(res_continuity, axis=0)

# Top 10 elementos con mayor error en momentum X
top_10_idx_momentum_x = np.argsort(max_res_momentum_x_per_elem)[-100:][::-1]
print("Top 10 elementos con mayor residuo en Momentum X:")
for i, idx in enumerate(top_10_idx_momentum_x):
    print(f"{i+1}. Elemento {idx}: {max_res_momentum_x_per_elem[idx]:.6e}")

print("\n" + "="*60 + "\n")

# Top 10 elementos con mayor error en momentum Y
top_10_idx_momentum_y = np.argsort(max_res_momentum_y_per_elem)[-100:][::-1]
print("Top 10 elementos con mayor residuo en Momentum Y:")
for i, idx in enumerate(top_10_idx_momentum_y):
    print(f"{i+1}. Elemento {idx}: {max_res_momentum_y_per_elem[idx]:.6e}")

print("\n" + "="*60 + "\n")

# Top 10 elementos con mayor error en momentum Y
top_10_idx_momentum_z = np.argsort(max_res_momentum_z_per_elem)[-100:][::-1]
print("Top 10 elementos con mayor residuo en Momentum Z:")
for i, idx in enumerate(top_10_idx_momentum_z):
    print(f"{i+1}. Elemento {idx}: {max_res_momentum_z_per_elem[idx]:.6e}")

print("\n" + "="*60 + "\n")

# Top 10 elementos con mayor error en continuidad
top_10_idx_continuity = np.argsort(max_res_continuity_per_elem)[-100:][::-1]
print("Top 10 elementos con mayor residuo en Continuidad:")
for i, idx in enumerate(top_10_idx_continuity):
    print(f"{i+1}. Elemento {idx}: {max_res_continuity_per_elem[idx]:.6e}")

Top 10 elementos con mayor residuo en Momentum X:
1. Elemento 1601: 4.522729e+02
2. Elemento 2082: 3.668950e+02
3. Elemento 573: 3.392419e+02
4. Elemento 5358: 3.237022e+02
5. Elemento 4037: 3.099653e+02
6. Elemento 3001: 2.975842e+02
7. Elemento 1616: 2.854973e+02
8. Elemento 4891: 2.590822e+02
9. Elemento 2368: 2.289820e+02
10. Elemento 3920: 2.279461e+02
11. Elemento 272: 2.246334e+02
12. Elemento 3363: 2.228908e+02
13. Elemento 4490: 2.012741e+02
14. Elemento 4038: 1.961097e+02
15. Elemento 4693: 1.956140e+02
16. Elemento 5812: 1.931993e+02
17. Elemento 2369: 1.749875e+02
18. Elemento 1871: 1.733627e+02
19. Elemento 443: 1.666711e+02
20. Elemento 329: 1.632431e+02
21. Elemento 183: 1.578753e+02
22. Elemento 790: 1.558159e+02
23. Elemento 413: 1.544511e+02
24. Elemento 6570: 1.502847e+02
25. Elemento 6661: 1.487104e+02
26. Elemento 6402: 1.471446e+02
27. Elemento 3594: 1.389996e+02
28. Elemento 523: 1.297134e+02
29. Elemento 395: 1.222820e+02
30. Elemento 1455: 1.118659e+02
31. Elem

In [22]:
# Índices de DOFs a analizar
index_dof = top_10_idx_momentum_z[:7]

# Obtener elementos y tetraedros
elements = mesh.t.T
nodes_coords = mesh.p

# Crear figura
fig = go.Figure()

# 1. Graficar la malla completa (semitransparente)
# Extraer todas las caras del boundary
boundary_facets_idx = mesh.boundary_facets()
boundary_faces = mesh.facets[:, boundary_facets_idx]

fig.add_trace(
    go.Mesh3d(
        x=nodes_coords[0, :],
        y=nodes_coords[1, :],
        z=nodes_coords[2, :],
        i=boundary_faces[0, :],
        j=boundary_faces[1, :],
        k=boundary_faces[2, :],
        color='lightgray',
        opacity=0.2,
        name='Malla completa',
        showlegend=True,
        lighting=dict(ambient=0.6, diffuse=0.5, specular=0.3)
    )
)

# 2. Para cada DOF, encontrar elementos que lo contienen
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta']

for idx, dof in enumerate(index_dof):
    # Encontrar elementos que contienen este DOF
    element_indices = np.where(basis_u.element_dofs == dof)[1]
    
    if len(element_indices) == 0:
        continue
    
    # Obtener tetraedros de esos elementos
    elements_subset = elements[element_indices]
    
    # Crear caras de los tetraedros para visualización
    # Cada tetraedro tiene 4 caras triangulares
    faces_i, faces_j, faces_k = [], [], []
    
    for tet in elements_subset:
        # Cara 1: nodos 0, 1, 2
        faces_i.extend([tet[0], tet[0], tet[0], tet[1]])
        faces_j.extend([tet[1], tet[1], tet[2], tet[2]])
        faces_k.extend([tet[2], tet[3], tet[3], tet[3]])
    
    # Graficar elementos que contienen el DOF
    fig.add_trace(
        go.Mesh3d(
            x=nodes_coords[0, :],
            y=nodes_coords[1, :],
            z=nodes_coords[2, :],
            i=faces_i,
            j=faces_j,
            k=faces_k,
            color=colors[idx % len(colors)],
            opacity=0.6,
            name=f'Soporte DOF {dof}',
            showlegend=True
        )
    )
    
    # 3. Marcar la ubicación exacta del DOF
    dof_loc = basis_u.doflocs[:, dof]
    
    fig.add_trace(
        go.Scatter3d(
            x=[dof_loc[0]],
            y=[dof_loc[1]],
            z=[dof_loc[2]],
            mode='markers',
            marker=dict(
                size=8,
                color=colors[idx % len(colors)],
                symbol='diamond',
                line=dict(color='black', width=2)
            ),
            name=f'DOF {dof}',
            showlegend=True
        )
    )

# Configurar layout con mayor ancho
fig.update_layout(
    title='Soporte local de DOFs en Malla 3D',
    width=800,
    height=600,
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()